In [55]:
import pandas as pd


import csv

with open("dataset/online_retail.csv") as f:
    
    reader = csv.reader(f)
    
    dataset = [[], [], [], [], [], [], [], []]
    labels = ['invoiceNo', 'stockCode', 'description', 'quantity', 'invoiceDate', 'unitPrice', 'customerId', 'country']
    next(reader)
    cancelled = 0
    for row in reader:
        if(row[0].startswith('C')) or len(row) != 8:
            cancelled+=1
            continue
        else:
            for i in range(8):
                dataset[i].append(row[i])
    
    print(labels)
    print(f"Cancelled invoices: {cancelled}")

['invoiceNo', 'stockCode', 'description', 'quantity', 'invoiceDate', 'unitPrice', 'customerId', 'country']
Cancelled invoices: 9288


In [56]:
dictionary = {}

for i, value in enumerate(dataset[0]):
    if value in dictionary.keys():
        dictionary[value].append(dataset[2][i])
    else:
        dictionary[value] = [dataset[2][i]]
        
print(f"Dictionary loaded. Number of different invoices: {len(dictionary.keys())}")
print(f"Items for invoiceNo 574021 are :{dictionary['574021']}")

Dictionary loaded. Number of different invoices: 22064
Items for invoiceNo 574021 are :['GARDENERS KNEELING PAD KEEP CALM ', 'HOT WATER BOTTLE KEEP CALM', 'DOORMAT KEEP CALM AND COME IN']


In [122]:
invoices_list = sorted([key for key in dictionary.keys()])
items_list = sorted(list(set(dataset[2])))
print(f"Chack the number of different invoices: {len(invoices_list)}")
print(f"Number of different items: {len(items_list)}")
print("-->", items_list[1])

Chack the number of different invoices: 22064
Number of different items: 4208
-->  4 PURPLE FLOCK DINNER CANDLES


### Building PA matrix


In [85]:
matrix = []
for invoice in invoices_list:
    item_list_for_invoice = []
    for item in items_list:
        if item in dictionary[invoice]: 
            item_list_for_invoice.append(1)
        else:
            item_list_for_invoice.append(0)
        
    matrix.append(item_list_for_invoice)
        

In [87]:


print(item_list_for_invoice)
print(len(matrix[1]))
print(entra)

[1, 1, 1, 1, 1, 1, 1]
4208
7


## Using pandas DF


In [107]:
df = pd.DataFrame(data = matrix, columns = items_list)

In [108]:
print(len(df))

22064


### FP-growth 

In [142]:
from mlxtend.frequent_patterns import fpgrowth, association_rules

fi = fpgrowth(df, 0.02)
print(len(fi))
print(fi.to_string())


303
      support            itemsets
0    0.102429              (3904)
1    0.020803              (1902)
2    0.021710              (1645)
3    0.065945               (244)
4    0.035216              (1738)
5    0.028508              (2026)
6    0.027194              (1076)
7    0.020350              (1283)
8    0.051351              (1825)
9    0.051033              (2743)
10   0.047634               (166)
11   0.044416               (162)
12   0.040790              (3506)
13   0.038479              (2897)
14   0.035488               (165)
15   0.033176              (3016)
16   0.032043              (2056)
17   0.027556              (3782)
18   0.052574              (2431)
19   0.049855              (3970)
20   0.044008              (3966)
21   0.024610              (3952)
22   0.033584              (2837)
23   0.029142              (1763)
24   0.070885              (2046)
25   0.059826              (2387)
26   0.055203              (1856)
27   0.053662              (1871)
28   0.042

In [178]:
#fi -->print the df ordered by support, so the most relevant is fi['itemsets'][0]
 #convert to the list the frozenset because is the only way to access the items
max_supp = 0
max_pos = -1.0
for i, value in enumerate(fi['support']):
    if value> max_supp:
        max_pos = i
        max_supp = value

    
print(f"The most useful item is : {items_list[list(fi['itemsets'][max_pos])[0]]} with a support: {fi['support'][max_pos]}")

The most useful item is : WHITE HANGING HEART T-LIGHT HOLDER with a support: 0.10242929659173314


In [144]:
#number of items in the FP-growth
counter = 0
for frozen_set in fi['itemsets']:
    counter += len(frozen_set)

In [145]:
print(f"The number of itemsets in the FP-growth solution are {len(fi)} and the number of items is {counter}")

The number of itemsets in the FP-growth solution are 303 and the number of items is 361


### Association rules

In [155]:
associations_df = association_rules(fi, metric = 'confidence', min_threshold = 0.7)
associations_df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(3003),(1599),0.048314,0.046003,0.034808,0.720450,15.661099,0.032585,3.412622
1,(1599),(3003),0.046003,0.048314,0.034808,0.756650,15.661099,0.032585,3.910774
2,(723),(2861),0.033675,0.046864,0.023658,0.702557,14.991511,0.022080,3.204436
3,(2656),(1599),0.034717,0.046003,0.028689,0.826371,17.963591,0.027092,5.494452
4,(2656),(3003),0.034717,0.048314,0.027148,0.781984,16.185462,0.025471,4.365218
5,"(2656, 3003)",(1599),0.027148,0.046003,0.024565,0.904841,19.669380,0.023316,10.025342
6,"(2656, 1599)",(3003),0.028689,0.048314,0.024565,0.856240,17.722404,0.023179,6.619970
7,"(3003, 1599)",(2656),0.034808,0.034717,0.024565,0.705729,20.327948,0.023356,3.280253
8,(2656),"(3003, 1599)",0.034717,0.034808,0.024565,0.707572,20.327948,0.023356,3.300613
9,(1450),(1451),0.034355,0.041380,0.024746,0.720317,17.407520,0.023325,3.427520


In [164]:
print("Association DataFrame ordered by 'confidence' column:")
associations_df.sort_values(by='confidence', ascending = False)


Association DataFrame ordered by 'confidence' column:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,"(2656, 3003)",(1599),0.027148,0.046003,0.024565,0.904841,19.669380,0.023316,10.025342
6,"(2656, 1599)",(3003),0.028689,0.048314,0.024565,0.856240,17.722404,0.023179,6.619970
3,(2656),(1599),0.034717,0.046003,0.028689,0.826371,17.963591,0.027092,5.494452
4,(2656),(3003),0.034717,0.048314,0.027148,0.781984,16.185462,0.025471,4.365218
1,(1599),(3003),0.046003,0.048314,0.034808,0.756650,15.661099,0.032585,3.910774
0,(3003),(1599),0.048314,0.046003,0.034808,0.720450,15.661099,0.032585,3.412622
9,(1450),(1451),0.034355,0.041380,0.024746,0.720317,17.407520,0.023325,3.427520
8,(2656),"(3003, 1599)",0.034717,0.034808,0.024565,0.707572,20.327948,0.023356,3.300613
7,"(3003, 1599)",(2656),0.034808,0.034717,0.024565,0.705729,20.327948,0.023356,3.280253
2,(723),(2861),0.033675,0.046864,0.023658,0.702557,14.991511,0.022080,3.204436


## 2.1.8 - Apriori

In [167]:
from mlxtend.frequent_patterns import apriori


ap = apriori(df, 0.02)

In [168]:
print(len(fi))
print(fi.to_string())

303
      support            itemsets
0    0.102429              (3904)
1    0.020803              (1902)
2    0.021710              (1645)
3    0.065945               (244)
4    0.035216              (1738)
5    0.028508              (2026)
6    0.027194              (1076)
7    0.020350              (1283)
8    0.051351              (1825)
9    0.051033              (2743)
10   0.047634               (166)
11   0.044416               (162)
12   0.040790              (3506)
13   0.038479              (2897)
14   0.035488               (165)
15   0.033176              (3016)
16   0.032043              (2056)
17   0.027556              (3782)
18   0.052574              (2431)
19   0.049855              (3970)
20   0.044008              (3966)
21   0.024610              (3952)
22   0.033584              (2837)
23   0.029142              (1763)
24   0.070885              (2046)
25   0.059826              (2387)
26   0.055203              (1856)
27   0.053662              (1871)
28   0.042

In [179]:
max_supp = 0
max_pos = -1.0
for i, value in enumerate(ap['support']):
    if value> max_supp:
        max_pos = i
        max_supp = value

    
print(f"The most useful item is : {items_list[list(ap['itemsets'][max_pos])[0]]} with a support: {ap['support'][max_pos]}")

The most useful item is : WHITE HANGING HEART T-LIGHT HOLDER with a support: 0.10242929659173314


In [188]:
import timeit




# number=1 means that it executes the function only once
apriori_time = timeit.timeit(lambda: apriori(df, 0.02), number=1)

fpgrowth_time = timeit.timeit(lambda: fpgrowth(df, 0.02), number = 1)


In [189]:
print(f"Time APRIORI: ", apriori_time, " seconds")
print(f"Time FP_growth: ", fpgrowth_time, " seconds")

Time APRIORI:  50.545171599998866  seconds
Time FP_growth:  4.428049699999974  seconds
